In [1]:
import requests

In [2]:
host = 'localhost'

In [3]:
assert requests.get('http://%s:4324/health'%host).status_code == 200

In [4]:
resp = requests.post('http://%s:4324/verify'%host, '{"signedCredentials":{"issuer":"a"}}')
print(resp.status_code)
assert resp.status_code == 400 
assert resp.content

400


In [25]:
import json
creds = json.loads('{"@context":["https://www.w3.org/2018/credentials/v1",{"name":"schema:name"}],"type":["VerifiableCredential"],"credentialSubject":{"type":"Person","name":"Tejash"},"issuanceDate":"2021-08-27T10:57:57.237Z","issuer":"did:authorizedIssuer:23423#21","proof":{"type":"Ed25519Signature2018","created":"2022-07-19T05:11:21Z","verificationMethod":"did:authorizedIssuer:23423#21","proofPurpose":"assertionMethod","jws":"eyJhbGciOiJFZERTQSIsImI2NCI6ZmFsc2UsImNyaXQiOlsiYjY0Il19..O6u5oAOiOeOt4nCHlJRRn04EyOoi7sRZHcJtvRXQqvKxQUfKdYjB9GJunO5pjnErWJC4U7gTctmUq34qrx98Bw"}}')
resp = requests.post('http://%s:4324/verify'%host, json={"signedCredentials":creds})
print(resp.status_code, resp.content)
assert resp.status_code==200
assert resp.json()["verified"]
resp.content
print("PASS")

200 b'{"verified":true,"results":[{"proof":{"@context":"https://w3id.org/security/v2","type":"Ed25519Signature2018","created":"2022-07-19T05:11:21Z","verificationMethod":"did:authorizedIssuer:23423#21","proofPurpose":"assertionMethod","jws":"eyJhbGciOiJFZERTQSIsImI2NCI6ZmFsc2UsImNyaXQiOlsiYjY0Il19..O6u5oAOiOeOt4nCHlJRRn04EyOoi7sRZHcJtvRXQqvKxQUfKdYjB9GJunO5pjnErWJC4U7gTctmUq34qrx98Bw"},"verified":true,"purposeResult":{"valid":true,"controller":{"@context":"https://w3id.org/security/v2","id":"did:authorizedIssuer:23423#21","publicKey":[{"@context":"https://w3id.org/security/v2","id":"did:authorizedIssuer:23423#21","type":"Ed25519VerificationKey2018","controller":"https://sunbird.org/","publicKeyBase58":"DaipNW4xaH2bh1XGNNdqjnSYyru3hLnUgTBSfSvmZ2hi"}],"assertionMethod":["did:authorizedIssuer:23423#21"]}}}]}'
PASS


In [67]:
resp = requests.post('http://%s:4324/sign'%host, json={"signedCredentials":""})
assert resp.status_code == 400

In [26]:
template="""{
        "@context": [
            "https://www.w3.org/2018/credentials/v1",
            {
                "@context": {
                    "@version": 1.1,
                    "@protected": true,
                    "SkillCertificate": {
                        "@id": "https://github.com/sunbird-specs/vc-specs#SkillCertificate",
                        "@context": {
                            "id": "@id",
                            "@version": 1.1,
                            "@protected": true,
                            "skills": {
                                "@id": "https://github.com/sunbird-specs/vc-specs#skills",
                                "@container": "@list"
                            },
                            "name":"schema:Text",
                            "refId":"schema:Text"
                        }
                    },
                    
                    "trainedOn": {
                        "@id": "https://github.com/sunbird-specs/vc-specs#trainedOn",
                        "@context": {
                            "name": "schema:Text"
                        }
                    }
                }
            }
        ],
        "type": [
            "VerifiableCredential","SkillCertificate"
        ],
        "credentialSubject": {
            "type": "SkillCertificate",
            "refId": "{{id}}",
            "name": "{{name}}",
            "skills": [{{#each skills}}"{{skill}}"{{#unless @last}},{{/unless}}{{/each}}]
        },
        "issuanceDate": "{{date}}",
        "issuer": "did:authorizedIssuer:23423#21",
        "evidence": [
            {
                "type": [
                    "Vaccination"
                ],
                "id": "https://sunbird.org/id"
            }
        ]
    }"""

In [27]:
entity = {
        "name": "a",
        "id": "123",
        "skills": [{"skill":"a"},{"skill":"b"}],
        "date":"2022-07-01T12:12:00Z"
    }

In [28]:
resp = requests.post('http://%s:4324/sign'%host, json={"data":entity, "credentialTemplate":template})
print('status', resp.status_code)
print(resp.content)
assert resp.status_code == 200
resp.json()['proof']['type'] == 'Ed25519Signature2018'

status 200
b'{"@context":["https://www.w3.org/2018/credentials/v1",{"@context":{"@version":1.1,"@protected":true,"SkillCertificate":{"@id":"https://github.com/sunbird-specs/vc-specs#SkillCertificate","@context":{"id":"@id","@version":1.1,"@protected":true,"skills":{"@id":"https://github.com/sunbird-specs/vc-specs#skills","@container":"@list"},"name":"schema:Text","refId":"schema:Text"}},"trainedOn":{"@id":"https://github.com/sunbird-specs/vc-specs#trainedOn","@context":{"name":"schema:Text"}}}}],"type":["VerifiableCredential","SkillCertificate"],"credentialSubject":{"type":"SkillCertificate","refId":"123","name":"a","skills":["a","b"]},"issuanceDate":"2022-07-01T12:12:00Z","issuer":"did:authorizedIssuer:23423#21","evidence":[{"type":["Vaccination"],"id":"https://sunbird.org/id"}],"proof":{"type":"Ed25519Signature2018","created":"2022-08-01T06:45:06Z","verificationMethod":"did:authorizedIssuer:23423#21","proofPurpose":"assertionMethod","jws":"eyJhbGciOiJFZERTQSIsImI2NCI6ZmFsc2UsImNyaXQi

True

In [72]:
resp.content

b'{"error": "Internal Server Error", "message": Dereferencing a URL did not result in a valid JSON-LD object. Possible causes are an inaccessible URL perhaps due to a same-origin policy (ensure the server uses CORS if you are using client-side JavaScript), too many redirects, a non-JSON response, or more than one HTTP Link Header was provided for a remote context.}'

In [61]:
assert resp.json()["@context"][0] == 'https://www.w3.org/2018/credentials/v1'